In [1]:
from __future__ import division
from past.utils import old_div
from proteus import FemTools as ft
from proteus import MeshTools as mt
import proteus.default_p as physics 
import proteus.default_n as numerics
from proteus.TransportCoefficients import PoissonEquationCoefficients


class Hi:
    def __init__(self): 
        pass
    def hello(self):
        return 'hello PotentialFlowProblem as PT'


class PotentialFlowProblem:
    """ PotentialFlowProblem """

    def __init__(self,
                 cfl=0.33,
                 # DOMAIN AND MESH #
                 domain=None,
                 mesh=None,
                 # INITIAL CONDITIONS #
                 initialConditions=None,
                 # BOUNDARY CONDITIONS #
                 boundaryConditions=None,
                 ):

        # ---assertions ---
        assert cfl <= 1, "Choose cfl <= 1"
        assert domain.nd in [2,3], "nd={2,3}"
        assert domain is not None, "Provide a domain"
        
        # --- save parameters ---
        self.domain = domain
        self.mesh = mesh
        
        # ***** CREATE SYSTEM PHYSICS OBJECT ***** #
        def r(x):
            return 0.
        def a(x):
            if self.domain.nd == 2:
                return np.array([[1.,0.],
                                 [0.,1.]])
            elif self.domain.nd == 3:
                return np.array([[1.,0.,0.],
                                 [0.,1.,0.],
                                 [0.,0.,1.]])
        physics.coefficients = PoissonEquationCoefficients([a],[r]) 
        physics.name="PoissonDots"

         # ***** CREATE SYSTEM PHYSICS OBJECT ***** #
        self.SystemPhysics = SystemPhysics(ProblemInstance=self)
        self.SystemPhysics.initialConditions = initialConditions
        self.SystemPhysics.boundaryConditions= boundaryConditions        
        
        # ***** CREATE SYSTEM NUMERICS OBJECT ***** #
        self.SystemNumerics = SystemNumerics(ProblemInstance=self)

        # ***** CREATE MODEL PARAMETERS OBJECT ***** #
        self.Parameters = Parameters.ParametersHolder(ProblemInstance=self)

        # ***** CREATE FINITE ELEMENT SPACES ***** #
        self.FESpace = FESpace(ProblemInstance=self)
        self.FESpace = FESpace().getFESpace()
        
        # ***** CREATING OUTPUT MANAGEMENT OBJECTS ***** #
        self.so = System_base()
        self.outputStepping=OutputStepping()

        


        # ***** DEFINE PHYSICAL AND NUMERICAL PARAMETERS ***** #
        self.physical_parameters = default_physical_parameters
        self.swe_parameters = default_swe_parameters
        self.GN_swe_parameters = default_GN_swe_parameters

        
        
class OutputStepping:
    """
    OutputStepping handles how often the solution is outputted.
    """
    def __init__(self):
        self.final_time=None 
        self.dt_init=0.001
        self.dt_output=None
        self.nDTout = None
        self.dt_fixed = None

    def __getitem__(self, key):
        return self.__dict__[key]

    def setOutputStepping(self):
        assert not (self.dt_output is None and nDTout is None), "Provide dt_output or nDTout"
        # COMPUTE dt_init #
        self.dt_init = min(self.dt_output, self.dt_init)
        if self.nDTout is None:
            self.nDTout = int(round(old_div(self.final_time, self.dt_output)))
        else:
            self.dt_output = float(self.final_time)/float(self.nDTout)

class FESpace:
    """
    Create FE Spaces.
    """

    def __init__(self):
        pass

    def getFESpace(self):
        basis = ft.C0_AffineLinearOnSimplexWithNodalBasis  # p1 space
        # QUADRATURE RULE #
        elementQuadrature = ft.SimplexGaussQuadrature(2, 3)
        elementBoundaryQuadrature = ft.SimplexGaussQuadrature(1, 3)
        return {'basis': basis,
                'elementQuadrature': elementQuadrature,
                'elementBoundaryQuadrature': elementBoundaryQuadrature}


# ***************************************** #
# ********** PHYSICAL PARAMETERS ********** #
# ***************************************** #
default_physical_parameters = {'gravity': 9.81,
                               'LINEAR_FRICTION': 0,
                               'mannings': 0.0,
                               'gen_start': 0.0,
                               'gen_length': 0.0,
                               'abs_start': 0.0,
                               'abs_length': 0.0,}

# ****************************************** #
# ********** NUMERICAL PARAMETERS ********** #
# ****************************************** #
default_swe_parameters = {'LUMPED_MASS_MATRIX': 0,
                          'cfl': 0.33,
                          'SSPOrder': 3,
                          'cE' : 1.}
default_GN_swe_parameters = {'LUMPED_MASS_MATRIX': 0,
                             'cfl': 0.33,
                             'SSPOrder': 3}
